In [1]:
from src.setup.mlflow_setup import init_mlflow

MLflow initialization module imported.


In [1]:
import os
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")

In [3]:
init_mlflow()

In [2]:
df = pd.read_csv('../data/raw/telecom.csv')

In [3]:
df['ratio_day_night_calls'] = round(df['total_day_calls'] / df['total_night_calls'], 2)
df['ratio_day_night_calls'] = df['ratio_day_night_calls'].replace([np.inf, -np.inf], 0)

In [4]:
df['ratio_day_night_calls'].value_counts()

ratio_day_night_calls
0.92    74
0.89    72
0.94    71
1.05    68
0.97    67
        ..
0.36     1
2.14     1
2.29     1
2.48     1
2.53     1
Name: count, Length: 206, dtype: int64

In [5]:
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,...,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,ratio_day_night_calls
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,...,16.62,254.4,103,11.45,13.7,3,3.70,1,no,1.19
1,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,...,16.62,254.4,103,11.45,13.7,3,3.70,1,no,1.19
2,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,...,10.30,162.6,104,7.32,12.2,5,3.29,0,no,1.10
3,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,...,5.26,196.9,89,8.86,6.6,7,1.78,2,no,0.80
4,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,...,12.61,186.9,121,8.41,10.1,3,2.73,3,no,0.93


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4251 entries, 0 to 4250
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          4251 non-null   object 
 1   account_length                 4251 non-null   int64  
 2   area_code                      4251 non-null   object 
 3   international_plan             4251 non-null   object 
 4   voice_mail_plan                4251 non-null   object 
 5   number_vmail_messages          4251 non-null   int64  
 6   total_day_minutes              4251 non-null   float64
 7   total_day_calls                4251 non-null   int64  
 8   total_day_charge               4251 non-null   float64
 9   total_eve_minutes              4251 non-null   float64
 10  total_eve_calls                4251 non-null   int64  
 11  total_eve_charge               4251 non-null   float64
 12  total_night_minutes            4251 non-null   f

## Stat EDA

In [11]:
from scipy import stats
from scipy.stats import shapiro, normaltest, jarque_bera
from scipy.stats import levene, bartlett

from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)

# Optional: statsmodels for Breusch–Pagan / White tests
try:
    import statsmodels.api as sm
    from statsmodels.stats.diagnostic import het_breuschpagan, het_white
    STATS_MODELS_OK = True
except Exception:
    STATS_MODELS_OK = False

In [12]:
na = df.isna().sum().sort_values(ascending=False)
display(na[na > 0])
print('Total missing:', int(df.isna().sum().sum()))
print('Duplicates:', int(df.duplicated().sum()))

Series([], dtype: int64)

Total missing: 0
Duplicates: 1


In [13]:
if df['churn'].dtype == 'object':
    churn_map = {'yes': 1, 'no': 0, 'true': 1, 'false': 0, '1': 1, '0': 0}
    churn_num = df['churn'].astype(str).str.strip().str.lower().map(churn_map)
    if churn_num.isna().any():
        churn_num = pd.Series(pd.factorize(df['churn'])[0], index=df.index)
else:
    churn_num = df['churn'].astype(float)

df = df.copy()
df['churn_num'] = churn_num

cat_cols = df.select_dtypes(include=['object']).columns.tolist()
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print('categorical:', cat_cols)
print('numeric:', num_cols)

categorical: ['state', 'area_code', 'international_plan', 'voice_mail_plan', 'churn']
numeric: ['account_length', 'number_vmail_messages', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 'total_eve_minutes', 'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 'total_night_calls', 'total_night_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls', 'ratio_day_night_calls', 'churn_num']


In [14]:
summary_num = df[num_cols].describe().T
summary_num['skew'] = df[num_cols].skew(numeric_only=True)
summary_num['kurtosis'] = df[num_cols].kurtosis(numeric_only=True)

display(summary_num.sort_values('skew', key=lambda s: s.abs(), ascending=False).head(15))

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis
churn_num,4251.0,0.140673,0.347725,0.0,0.000,0.00,0.000,1.00,2.067707,2.276485
number_vmail_messages,4251.0,7.636086,13.441254,0.0,0.000,0.00,16.000,52.00,1.371984,0.269986
total_intl_calls,4251.0,4.426017,2.462876,0.0,3.000,4.00,6.000,20.00,1.360484,3.264480
ratio_day_night_calls,4251.0,1.046349,0.326073,0.0,0.830,1.00,1.220,3.94,1.307060,4.611113
number_customer_service_calls,4251.0,1.558927,1.311307,0.0,1.000,1.00,2.000,9.00,1.083032,1.656758
total_intl_charge,4251.0,2.769873,0.745253,0.0,2.300,2.78,3.240,5.40,-0.241990,0.702332
total_intl_minutes,4251.0,10.256881,2.760282,0.0,8.500,10.30,12.000,20.00,-0.241679,0.701963
account_length,4251.0,100.237826,39.693865,1.0,73.000,100.00,127.000,243.00,0.122221,-0.131558
total_day_calls,4251.0,99.912726,19.851642,0.0,87.000,100.00,113.000,165.00,-0.086232,0.192836
total_eve_minutes,4251.0,200.172806,50.243657,0.0,165.950,200.70,233.750,359.30,-0.030353,0.044154


In [15]:
churn_counts = df['churn'].value_counts(dropna=False)
churn_rate = df['churn_num'].mean()
print('churn counts:\n', churn_counts)
print('churn rate (mean of churn_num):', float(churn_rate))

churn counts:
 churn
no     3653
yes     598
Name: count, dtype: int64
churn rate (mean of churn_num): 0.14067278287461774


In [16]:
outlier_rows = []

for col in [c for c in num_cols if c != 'churn_num']:
    x = df[col].dropna()
    if len(x) == 0:
        continue

    # IQR
    q1, q3 = x.quantile(0.25), x.quantile(0.75)
    iqr = q3 - q1
    lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
    iqr_n = int(((x < lo) | (x > hi)).sum())

    # Z-score
    z = np.abs(stats.zscore(x, nan_policy='omit'))
    z_n = int((z > 3).sum())

    # MAD (modified Z)
    med = np.median(x)
    mad = np.median(np.abs(x - med))
    if mad == 0:
        mz_n = 0
    else:
        mz = 0.6745 * (x - med) / mad
        mz_n = int((np.abs(mz) > 3.5).sum())

    outlier_rows.append({
        'feature': col,
        'n': int(x.shape[0]),
        'iqr_outliers': iqr_n,
        'iqr_pct': 100*iqr_n/len(x),
        'z_outliers': z_n,
        'z_pct': 100*z_n/len(x),
        'mad_outliers': mz_n,
        'mad_pct': 100*mz_n/len(x),
        'iqr_low': float(lo),
        'iqr_high': float(hi),
    })

outliers = pd.DataFrame(outlier_rows).sort_values('iqr_pct', ascending=False)
display(outliers.head(20))

,feature,n,iqr_outliers,iqr_pct,z_outliers,z_pct,mad_outliers,mad_pct,iqr_low,iqr_high
14,number_customer_service_calls,4251,335,7.880499,45,1.058574,17,0.399906,-0.5000,3.5000
15,ratio_day_night_calls,4251,114,2.681722,57,1.340861,62,1.458480,0.2450,1.8050
12,total_intl_calls,4251,100,2.352388,62,1.458480,159,3.740296,-1.5000,10.5000
1,number_vmail_messages,4251,86,2.023053,10,0.235239,0,0.000000,-24.0000,40.0000
11,total_intl_minutes,4251,62,1.458480,32,0.752764,26,0.611621,3.2500,17.2500
13,total_intl_charge,4251,62,1.458480,32,0.752764,26,0.611621,0.8900,4.6500
8,total_night_minutes,4251,37,0.870383,16,0.376382,6,0.141143,66.0750,335.8750
10,total_night_charge,4251,37,0.870383,16,0.376382,6,0.141143,2.9725,15.1125
5,total_eve_minutes,4251,34,0.799812,13,0.305810,2,0.047048,64.2500,335.4500
7,total_eve_charge,4251,34,0.799812,13,0.305810,2,0.047048,5.4650,28.5050


In [19]:
normality_rows = []

for col in [c for c in num_cols if c != 'churn_num']:
    x = df[col].dropna()
    if len(x) < 8:
        continue

    if len(x) <= 5000:
        sh_stat, sh_p = shapiro(x)
    else:
        sh_stat, sh_p = np.nan, np.nan

    da_stat, da_p = normaltest(x)
    jb_stat, jb_p = jarque_bera(x)

    normality_rows.append({
        'feature': col,
        'n': int(len(x)),
        'shapiro_p': sh_p,
        'dagostino_p': da_p,
        'jarque_bera_p': jb_p,
        'skew': float(stats.skew(x)),
        'kurtosis': float(stats.kurtosis(x)),
    })

normality = pd.DataFrame(normality_rows)
normality['non_normal'] = (normality['dagostino_p'] < 0.05) | (normality['jarque_bera_p'] < 0.05)
display(normality.sort_values('skew', key=lambda s: s.abs(), ascending=False).head(20))

,feature,n,shapiro_p,dagostino_p,jarque_bera_p,skew,kurtosis,non_normal
1,number_vmail_messages,4251,2.349192e-71,1.073352e-174,6.933013e-293,1.371499,0.268258,True
12,total_intl_calls,4251,5.454772e-46,5.434590e-242,0.000000e+00,1.360004,3.259231,True
15,ratio_day_night_calls,4251,7.471360e-40,6.889436e-258,0.000000e+00,1.306598,4.604280,True
14,number_customer_service_calls,4251,7.778635e-50,5.790468e-159,3.340499e-286,1.082650,1.653399,True
13,total_intl_charge,4251,1.782838e-13,2.317604e-20,1.398081e-28,-0.241904,0.700095,True
11,total_intl_minutes,4251,1.897143e-13,2.486390e-20,1.543308e-28,-0.241594,0.699727,True
0,account_length,4251,2.350869e-05,9.439911e-04,1.059146e-03,0.122178,-0.132815,True
3,total_day_calls,4251,2.265842e-04,4.150590e-03,2.823138e-03,-0.086202,0.191198,True
5,total_eve_minutes,4251,5.258895e-01,5.971721e-01,6.141336e-01,-0.030342,0.042691,False
7,total_eve_charge,4251,5.253869e-01,5.980896e-01,6.150539e-01,-0.030316,0.042568,False


In [20]:
corr = df[num_cols].corr(numeric_only=True)

pairs = []
cols = corr.columns
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        r = corr.iloc[i, j]
        if pd.notna(r) and abs(r) >= 0.7:
            pairs.append({'f1': cols[i], 'f2': cols[j], 'r': float(r)})

high_corr = pd.DataFrame(pairs).sort_values('r', key=lambda s: s.abs(), ascending=False)
display(high_corr)

,f1,f2,r
0,total_day_minutes,total_day_charge,1.000000
1,total_eve_minutes,total_eve_charge,1.000000
2,total_night_minutes,total_night_charge,0.999999
4,total_intl_minutes,total_intl_charge,0.999993
3,total_night_calls,ratio_day_night_calls,-0.700402


In [21]:
resid_rows = []

for col in [c for c in num_cols if c not in ['churn_num']]:
    x = df[[col]].dropna()
    if len(x) < 10:
        continue

    y = df.loc[x.index, 'churn_num'].astype(float).values
    X = x.values

    lr = LinearRegression().fit(X, y)
    y_hat = lr.predict(X)
    resid = y - y_hat

    dw = np.sum(np.diff(resid)**2) / np.sum(resid**2) if np.sum(resid**2) > 0 else np.nan

    resid_rows.append({
        'feature': col,
        'coef': float(lr.coef_.ravel()[0]),
        'intercept': float(lr.intercept_),
        'resid_mean': float(np.mean(resid)),
        'resid_std': float(np.std(resid)),
        'resid_skew': float(stats.skew(resid)),
        'resid_kurtosis': float(stats.kurtosis(resid)),
        'durbin_watson': float(dw),
    })

residuals = pd.DataFrame(resid_rows)
residuals['dw_flag'] = (residuals['durbin_watson'] < 1.5) | (residuals['durbin_watson'] > 2.5)
display(residuals.sort_values('resid_skew', key=lambda s: s.abs(), ascending=False).head(20))

,feature,coef,intercept,resid_mean,resid_std,resid_skew,resid_kurtosis,durbin_watson,dw_flag
6,total_eve_calls,-0.000119,0.152625,1.211817e-17,0.347676,2.066832,2.272181,2.004245,False
3,total_day_calls,0.000202,0.120498,4.178680e-19,0.347661,2.066585,2.271871,2.004124,False
9,total_night_calls,-0.000220,0.162642,2.089340e-18,0.347656,2.066480,2.271675,2.004922,False
0,account_length,0.000170,0.123671,5.432284e-18,0.347619,2.065807,2.270659,2.005822,False
15,ratio_day_night_calls,0.022093,0.117556,6.268020e-19,0.347609,2.065716,2.270776,2.004855,False
12,total_intl_calls,-0.004839,0.162092,2.548995e-17,0.347479,2.063587,2.267913,2.004019,False
10,total_night_charge,0.007141,0.076233,5.432284e-18,0.347307,2.060125,2.261990,2.004126,False
8,total_night_minutes,0.000321,0.076225,6.268020e-19,0.347307,2.060124,2.261987,2.004126,False
13,total_intl_charge,0.025684,0.069531,-5.850152e-18,0.347157,2.057728,2.258982,2.009049,False
11,total_intl_minutes,0.006936,0.069535,-1.253604e-18,0.347156,2.057725,2.258981,2.009043,False


In [23]:
if STATS_MODELS_OK:
    bp_rows = []
    for col in [c for c in num_cols if c not in ['churn_num']]:
        x = df[[col]].dropna()
        if len(x) < 30:
            continue

        y = df.loc[x.index, 'churn_num'].astype(float)
        X = sm.add_constant(x.astype(float))

        ols = sm.OLS(y, X).fit()

        bp_lm, bp_lm_p, bp_f, bp_f_p = het_breuschpagan(ols.resid, ols.model.exog)
        w_lm, w_lm_p, w_f, w_f_p = het_white(ols.resid, ols.model.exog)

        bp_rows.append({
            'feature': col,
            'bp_lm_p': float(bp_lm_p),
            'bp_f_p': float(bp_f_p),
            'white_lm_p': float(w_lm_p),
            'white_f_p': float(w_f_p),
            'hetero_flag_bp': bool(bp_lm_p < 0.05),
        })

    hetero_bp = pd.DataFrame(bp_rows).sort_values('bp_lm_p')
    display(hetero_bp.head(20))
else:
    print('statsmodels not available; skipped Breusch–Pagan/White tests')

,feature,bp_lm_p,bp_f_p,white_lm_p,white_f_p,hetero_flag_bp
14,number_customer_service_calls,1.134517e-21,7.041014e-22,3.795477e-27,1.631939e-27,True
2,total_day_minutes,3.565565e-21,2.266719e-21,1.178515e-57,1.912821e-59,True
4,total_day_charge,3.588439e-21,2.281563e-21,1.181892e-57,1.918659e-59,True
1,number_vmail_messages,5.842961e-12,5.174890e-12,1.322881e-14,1.061700e-14,True
5,total_eve_minutes,5.094086e-07,4.937348e-07,1.613155e-06,1.561447e-06,True
7,total_eve_charge,5.099898e-07,4.943012e-07,1.615225e-06,1.563462e-06,True
11,total_intl_minutes,4.255267e-04,4.230601e-04,1.344336e-03,1.336786e-03,True
13,total_intl_charge,4.260414e-04,4.235730e-04,1.349828e-03,1.342261e-03,True
8,total_night_minutes,1.960987e-03,1.955462e-03,5.844999e-03,5.829840e-03,True
10,total_night_charge,1.963846e-03,1.958318e-03,5.855301e-03,5.840133e-03,True


In [24]:
print('Top outlier features (IQR %):')
display(outliers[['feature','iqr_pct','z_pct','mad_pct']].head(10))

print('High correlations (|r|>=0.7):')
display(high_corr)

print('Heteroscedasticity candidates (Levene p<0.05):')
display(hetero_quart[hetero_quart['hetero_flag_levene']].head(20))

Top outlier features (IQR %):


,feature,iqr_pct,z_pct,mad_pct
14,number_customer_service_calls,7.880499,1.058574,0.399906
15,ratio_day_night_calls,2.681722,1.340861,1.458480
12,total_intl_calls,2.352388,1.458480,3.740296
1,number_vmail_messages,2.023053,0.235239,0.000000
11,total_intl_minutes,1.458480,0.752764,0.611621
13,total_intl_charge,1.458480,0.752764,0.611621
8,total_night_minutes,0.870383,0.376382,0.141143
10,total_night_charge,0.870383,0.376382,0.141143
5,total_eve_minutes,0.799812,0.305810,0.047048
7,total_eve_charge,0.799812,0.305810,0.047048


High correlations (|r|>=0.7):


,f1,f2,r
0,total_day_minutes,total_day_charge,1.000000
1,total_eve_minutes,total_eve_charge,1.000000
2,total_night_minutes,total_night_charge,0.999999
4,total_intl_minutes,total_intl_charge,0.999993
3,total_night_calls,ratio_day_night_calls,-0.700402


Heteroscedasticity candidates (Levene p<0.05):


,feature,levene_p,bartlett_p,hetero_flag_levene
4,total_day_charge,9.588695e-27,2.280314e-81,True
2,total_day_minutes,9.600339e-27,2.268528e-81,True
1,number_vmail_messages,2.791537e-07,5.591521e-06,True
14,number_customer_service_calls,6.325958e-06,9.859103e-28,True
12,total_intl_calls,4.595710e-05,2.628351e-07,True
7,total_eve_charge,1.454097e-02,6.546559e-01,True
5,total_eve_minutes,1.454415e-02,6.547618e-01,True


## Обучение модели

In [ ]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

In [59]:
X = df.drop(['churn'], axis=1)

In [60]:
y = df['churn']

In [61]:
y = y.map({'no': 0, 'yes': 1})

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [63]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [64]:
numeric = df.select_dtypes(include='number').columns

In [65]:
categorical = df.select_dtypes(exclude='number').columns

In [70]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric),
        ('cat', categorical_transformer, categorical.drop('churn'))
    ])

pipeline_weighted = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression())])
                        # ('classifier', LogisticRegression(class_weight='balanced', solver='liblinear'))])

pipeline_weighted.fit(X_train, y_train)


y_pred = pipeline_weighted.predict(X_test)

cm = classification_report(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1205
           1       0.58      0.25      0.35       198

    accuracy                           0.87      1403
   macro avg       0.74      0.61      0.64      1403
weighted avg       0.84      0.87      0.85      1403



In [ ]:
mlflow.set_experiment("churn_classification")      # имя эксперимента

with mlflow.start_run():

    # --- 1. Логируем параметры модели ---
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("solver", "liblinear")
    mlflow.log_param("class_weight", "balanced")

    # --- 2. Делаем предсказание ---
    y_pred = pipeline_weighted.predict(X_test)

    # --- 3. Метрики ---
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_score", f1)

    # можно логировать текстовый classification_report как артефакт
    report = classification_report(y_test, y_pred)
    with open("cls_report.txt", "w") as f:
        f.write(report)
    mlflow.log_artifact("cls_report.txt")

    # --- 4. Логируем сам PIPELINE ---
    mlflow.sklearn.log_model(
        sk_model=pipeline_weighted,
        name="model",
        registered_model_name="ChurnPipeline"  # опционально
    )

    print("Model logged to MLflow!")

2025/11/16 22:14:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ChurnPipeline' already exists. Creating a new version of this model...
2025/11/16 22:14:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ChurnPipeline, version 12
Created version '12' of model 'ChurnPipeline'.


Model logged to MLflow!
🏃 View run persistent-snail-80 at: http://94.228.117.198:5000/#/experiments/1/runs/3a5151f6410e4dc48bdbf865bf6b7c32
🧪 View experiment at: http://94.228.117.198:5000/#/experiments/1
